# 模型儲存

In [1]:
import tensorflow as tf

In [2]:
x_ = tf.placeholder(tf.float32, [None, 784], name="x_")
y_ = tf.placeholder(tf.float32, [None, 10], name="y_")


with tf.variable_scope("wb"):
    W = tf.get_variable("w", [784, 10],
        initializer=tf.random_normal_initializer())
    b = tf.get_variable("b", [10],
        initializer=tf.random_normal_initializer())
    y = tf.nn.softmax(tf.matmul(x_, W) + b)


with tf.variable_scope("cross_entropy") as scope:
    cross_entropy = -tf.reduce_sum(y_ * tf.log(y))

with tf.variable_scope("trainer") as scope:
    optimizer = tf.train.GradientDescentOptimizer(0.01)
    trainer = optimizer.minimize(cross_entropy)

with tf.variable_scope("evaluator") as scope:
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))



## global Step

In [3]:
global_step = tf.Variable(0, name="global_step", trainable=False)

## saver

In [4]:
saver = tf.train.Saver()

In [5]:
import input_data

mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [6]:
with tf.name_scope("initializer") as scope:
    init = tf.initialize_all_variables()

sess = tf.Session()
sess.run(init)


## add to collection

In [7]:
tf.add_to_collection('trainer', trainer)
tf.add_to_collection('accuracy', accuracy)
tf.add_to_collection('x_', x_)
tf.add_to_collection('y_', y_)
tf.add_to_collection('global_step', global_step)


In [8]:
#meta_graph_def = tf.train.export_meta_graph(filename='my_model.meta')

final_step = 1001
for i in range(sess.run(global_step), final_step):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(trainer,  feed_dict={x_: batch_xs, y_: batch_ys})
    sess.run(tf.assign(global_step, i))
    if i%20 == 0:
        print "%s, %s" %(i, (sess.run(accuracy, feed_dict={x_: mnist.test.images, y_: mnist.test.labels})))
        saver.save(sess, 'my_model.ckpt', global_step=global_step)
        if i == 200:
            break

FailedPreconditionError: Attempting to use uninitialized value global_step
	 [[Node: _send_global_step_0 = _Send[T=DT_INT32, client_terminated=true, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/cpu:0", send_device_incarnation=-4728940714305155713, tensor_name="global_step:0", _device="/job:localhost/replica:0/task:0/cpu:0"](global_step)]]